In [2]:
import pandas as pd
import numpy as np
import polars as pl
import fastexcel

In [3]:
# Read in file
df = pl.read_excel("data/Population estimates and components by SA2.xlsx", 
                   sheet_name="Table 2",
                   read_options={
                    "skip_rows": 6,
                    }
                   )
# Column names change
df.columns = ["GCCSA code",	"GCCSA name", "SA4 code", "SA4 name", "SA3 code",
              "SA3 name", "SA2 code", "SA2 name","2023 Pop", "2024 Pop",
              "2023-24 Change", "2023-24 Change %", "Natural Increase", "Net internal migration",
              "Net oveseas migration","Area","Population Density"]

In [20]:
pl.Config.set_tbl_rows(30)
df_stg = (
    df
        .select("SA3 name","SA3 code","SA2 name", "SA2 code", "2024 Pop", "2023-24 Change", "Area")
        .head(df.height - 2)
        .with_columns(
            pl.when(pl.col("SA2 name").str.contains("-"))
            .then(
                pl.col("SA2 name")
                    .str.extract(r"^(.*?)-", 1)
                    .str.strip_chars(" ")  # remove leading/trailing spaces
                )
            .otherwise(pl.col("SA2 name"))
            .alias("Suburb")
        )
        .with_columns(
            pl.when(pl.col("SA3 name").str.contains("-"))
            .then(
                pl.col("SA3 name")
                    .str.extract(r"^(.*?)-", 1)
                    .str.strip_chars(" ")  # remove leading/trailing spaces
                )
            .otherwise(pl.col("SA3 name"))
            .alias("Council")
        )
        .select("Council","Suburb","2024 Pop","2023-24 Change","Area")

        .group_by(["Council","Suburb"])
        .agg(
            pl.sum("2024 Pop").alias("2024 Pop"),
            pl.sum("2023-24 Change").alias("2023-24 Change"),
            pl.sum("Area").alias("Suburb Area km^2")
        )
        # Add average pop
        .with_columns(
            pl.col("2024 Pop")
            .mean()
            .over("Council")
            .round(0).cast(pl.Int64) 
            .alias("avg_suburb_pop_in_Council")
        )
        # Add average area
        .with_columns(
            pl.col("Suburb Area km^2")
            .mean()
            .over("Council")
            .round(0).cast(pl.Int64) 
            .alias("avg_suburb_area_in_Council")
        )
        .sort("Council", descending=False)

        .select("Council","Suburb","2024 Pop","avg_suburb_pop_in_Council",
                "Suburb Area km^2","avg_suburb_area_in_Council",
                "2023-24 Change")
        
)

In [29]:
int(df_stg.filter(pl.col("Council") == 'Manningham').select(pl.col('2024 Pop').mean()).item())

18831

In [21]:
df_stg

Council,Suburb,2024 Pop,avg_suburb_pop_in_Council,Suburb Area km^2,avg_suburb_area_in_Council,2023-24 Change
str,str,i64,i64,f64,i64,i64
"""Ballarat""","""Sebastopol""",12839,12356,10.6,45,-44
"""Ballarat""","""Smythes Creek""",4278,12356,104.7,45,11
"""Ballarat""","""Buninyong""",7358,12356,51.6,45,35
"""Ballarat""","""Delacombe""",14271,12356,34.2,45,1406
"""Ballarat""","""Ballarat""",11773,12356,12.4,45,-38
"""Ballarat""","""Alfredton""",20130,12356,52.7,45,1135
"""Ballarat""","""Canadian""",12457,12356,22.3,45,24
"""Ballarat""","""Wendouree""",15570,12356,67.6,45,140
"""Ballarat""","""Ballarat North""",15169,12356,73.1,45,156
